In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicConv2d(nn.Module):
    def __init__(self, max_in_channels, max_out_channels):
        super().__init__()
        self.weight = nn.Parameter(
            torch.randn(max_out_channels, max_in_channels, 1, 1)
        )
        self.bias = nn.Parameter(torch.zeros(max_out_channels))
        
    def forward(self, x, out_channels, in_channels=None):
        # 如果没指定输入通道数，就用x的实际通道数
        if in_channels is None:
            in_channels = x.size(1)
        # 选择需要的权重子集：out_channels行，in_channels列
        weight = self.weight[:out_channels, :in_channels, :, :]
        bias = self.bias[:out_channels]
        return F.conv2d(x, weight, bias)

class DynamicFPN(nn.Module):
    def __init__(self, in_channels_list=[1024, 2048], max_out_channels=256):
        super().__init__()
        self.channel_choices = [64, 128, 256]
        
        # 横向连接
        self.lateral_convs = nn.ModuleList([
            DynamicConv2d(in_c, max_out_channels)
            for in_c in in_channels_list
        ])
        
        # 自顶向下路径
        self.down_conv = DynamicConv2d(max_out_channels, max_out_channels)
        
    def forward(self, features, arch_config=[64, 256]):
        """
        features: [c4, c5] 
        arch_config: [p4通道数, p5通道数]
        """
        # 横向连接
        laterals = []
        for i, feat in enumerate(features):
            lat = self.lateral_convs[i](feat, out_channels=arch_config[i])
            laterals.append(lat)
        
        # 自顶向下路径
        out_feats = [laterals[-1]]  # p5
        
        # p5调整通道数匹配p4
        # 关键修改：指定输入和输出通道数
        p5_down = self.down_conv(
            out_feats[0], 
            out_channels=arch_config[0],  # 输出通道数(p4的通道数)
            in_channels=arch_config[1]    # 输入通道数(p5的通道数)
        )
        
        # 上采样
        p5_up = F.interpolate(p5_down, 
                            size=laterals[0].shape[-2:],
                            mode='nearest')
                            
        # 特征融合
        p4 = laterals[0] + p5_up
        
        return [p4, out_feats[0]]

# 测试
fpn = DynamicFPN()
c4 = torch.randn(1, 1024, 32, 32)
c5 = torch.randn(1, 2048, 16, 16)

# 随机采样两个不同的架构测试
out1 = fpn([c4, c5], [192, 256])    # p4:64通道, p5:256通道
print(out1[0].shape, out1[1].shape)
out2 = fpn([c4, c5], [128, 64])   # p4和p5都是128通道
print(out2[0].shape, out2[1].shape)